In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import argparse
import logging
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import f1_score, classification_report
from torch.utils.data import DataLoader, TensorDataset
import sys
from tqdm import tqdm, trange
import os
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import CyclicLR


/home/student/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import RobertaModel
import torch
import torch.nn as nn
from transformers.modeling_outputs import SequenceClassifierOutput

class RobertaWithMultiSampleDropout(nn.Module):
    def __init__(self, model_name, num_labels, dropout_rate=0.3, num_dropouts=5, use_multi_sample_dropout=True):
        super(RobertaWithMultiSampleDropout, self).__init__()
        self.roberta = RobertaModel.from_pretrained(model_name)
        self.use_multi_sample_dropout = use_multi_sample_dropout

        if self.use_multi_sample_dropout:
            self.dropouts = nn.ModuleList([
                nn.Dropout(dropout_rate) for _ in range(num_dropouts)
            ])
        else:
            self.dropout = nn.Dropout(dropout_rate)

        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[0]  # последний скрытый слой

        if self.use_multi_sample_dropout:
            logits_list = []
            for dropout in self.dropouts:
                dropped = dropout(sequence_output[:, 0, :])  # [CLS]-токен
                logits_list.append(self.classifier(dropped))
            avg_logits = torch.mean(torch.stack(logits_list), dim=0)
        else:
            dropped = self.dropout(sequence_output[:, 0, :])  # [CLS]-токен
            avg_logits = self.classifier(dropped)

        return SequenceClassifierOutput(logits=avg_logits)


In [3]:
def save_metrics_best(epoch, all_labels, all_preds, result_path, backprop, loss):
    import os
    import pandas as pd
    from sklearn.metrics import classification_report

    metrics_file = os.path.join(result_path, "metrics_best.csv")
    report = classification_report(all_labels, all_preds, output_dict=True)

    # Создаем словарь с метриками
    metrics_data = {
        "epoch": epoch,
        "accuracy": report["accuracy"],
        "macro_precision": report["macro avg"]["precision"],
        "macro_recall": report["macro avg"]["recall"],
        "macro_f1": report["macro avg"]["f1-score"],
        "weighted_precision": report["weighted avg"]["precision"],
        "weighted_recall": report["weighted avg"]["recall"],
        "weighted_f1": report["weighted avg"]["f1-score"],
        "backprop": backprop,
        "loss": loss
    }

    # Добавляем метрики по каждому классу
    for label in sorted(report.keys()):
        if isinstance(report[label], dict):
            metrics_data[f"class_{label}_precision"] = report[label]["precision"]
            metrics_data[f"class_{label}_recall"] = report[label]["recall"]
            metrics_data[f"class_{label}_f1"] = report[label]["f1-score"]

    # Создаем DataFrame и сохраняем его в CSV (перезапись файла)
    metrics_df = pd.DataFrame([metrics_data])
    metrics_df.to_csv(metrics_file, mode='w', header=True, index=False)

In [4]:
def compute_ens_weights(train_labels, beta: float = 0.999):
    train_labels = np.array(train_labels)
    classes, counts = np.unique(train_labels, return_counts=True)
    effective_num = (1 - np.power(beta, counts)) / (1 - beta)
    weights = 1.0 / effective_num
    weights = weights / np.sum(weights) * len(classes)  # нормализация как в оригинальной статье
    return torch.tensor(weights, dtype=torch.float)

In [5]:
def get_entity_tags_from_files(*file_paths):
    all_tags = set()
    for path in file_paths:
        try:
            df = pd.read_csv(path, sep='\t')
            if "entity_tag" in df.columns:
                tags = df["entity_tag"].dropna().unique().tolist()
                all_tags.update(tags)
        except Exception as e:
            print(f"[Ошибка] Не удалось загрузить {path}: {e}")
    return sorted(list(all_tags))

In [6]:
def load_data(file_path, tokenizer, max_seq_len):
    try:
        df = pd.read_csv(file_path, sep='\t')  # Загружаем CSV (TSV)

        # Проверка нужных колонок
        required_columns = {
            "sentence", "entity", "label", "entity_tag",
            "entity_pos_start_rel", "entity_pos_end_rel"
        }
        if not required_columns.issubset(df.columns):
            raise ValueError(f"Ожидаемые колонки: {required_columns}, но в файле: {df.columns}")

        # Заменяем -1 на 2 (если есть)
        df["label"] = df["label"].replace(-1, 2)

        # Размечаем предложения с [ENTITY] токенами
        def mark_entity_inline(row):
            sent = row["sentence"]
            start = row["entity_pos_start_rel"]
            end = row["entity_pos_end_rel"]
            tag = row["entity_tag"]
            tag_token = f"[ENTITY:{tag}]"
            tag_token_close = f"[/ENTITY:{tag}]"
            return (
                sent[:start] +
                f"{tag_token} " + sent[start:end] + f" {tag_token_close}" +
                sent[end:]
            )

        df["input_text"] = df.apply(mark_entity_inline, axis=1)
        texts = df["input_text"].tolist()
        labels = torch.tensor(df["label"].astype(int).tolist(), dtype=torch.long)

        # Токенизируем
        encodings = tokenizer(texts, padding=True, truncation=True,
                              max_length=max_seq_len, return_tensors='pt')
        dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)
        return dataset

    except FileNotFoundError:
        logging.error(f"Файл {file_path} не найден!")
        return None

In [7]:

def save_metrics(epoch, all_labels, all_preds, result_path, backprop, loss):
    metrics_file = os.path.join(result_path, "metrics_class_weights.csv")
    report = classification_report(all_labels, all_preds, output_dict=True)
    
    # Создаем DataFrame для метрик
    metrics_data = {
        "epoch": epoch,
        "accuracy": report["accuracy"],
        "macro_precision": report["macro avg"]["precision"],
        "macro_recall": report["macro avg"]["recall"],
        "macro_f1": report["macro avg"]["f1-score"],
        "weighted_precision": report["weighted avg"]["precision"],
        "weighted_recall": report["weighted avg"]["recall"],
        "weighted_f1": report["weighted avg"]["f1-score"],
        "backprop": backprop , # (Добавлено)
        "loss": loss
    }
    
    # Добавляем метрики для каждого класса
    for label in sorted(report.keys()):
        if isinstance(report[label], dict):  # Пропускаем 'accuracy', так как это float
            metrics_data[f"class_{label}_precision"] = report[label]["precision"]
            metrics_data[f"class_{label}_recall"] = report[label]["recall"]
            metrics_data[f"class_{label}_f1"] = report[label]["f1-score"]
    
    # Создаем DataFrame и сохраняем в CSV
    metrics_df = pd.DataFrame([metrics_data])
    metrics_df.to_csv(metrics_file, mode='a', header=not os.path.exists(metrics_file), index=False)

def save_confusion_matrix(epoch, y_true, y_pred, result_path,backprop):
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, columns=["Pred_0", "Pred_1", "Pred_2"], index=["True_0", "True_1", "True_2"])
    cm_file = os.path.join(result_path, f"confusion_matrix_class_weights_epoch_{epoch}_backprop_{backprop}.csv")
    cm_df.to_csv(cm_file)



In [8]:
from sklearn.metrics import classification_report
import pandas as pd

def castom_classification_report(all_labels, all_preds):
    report = classification_report(all_labels, all_preds, output_dict=True)
    data = []
    for label, metrics in report.items():
        if isinstance(metrics, dict):  # Для всех метрик, кроме accuracy
            row = {'label': label}
            for metric, value in metrics.items():
                row[metric] = round(value * 100, 3) if metric != 'support' else value
            data.append(row)
        else:  # Для accuracy
            data.append({'label': 'accuracy', 'precision': round(metrics * 100, 3), 'recall': None, 'f1-score': None, 'support': None})

    # Добавляем macro F1 для классов 1 и 2
    if '1' in report and '2' in report:
        f1_1 = report['1']['f1-score']
        f1_2 = report['2']['f1-score']
        f1_macro_1_2 = (f1_1 + f1_2) / 2
        data.append({
            'label': 'avg f1 (class 1&2)',
            'precision': None,
            'recall': None,
            'f1-score': round(f1_macro_1_2 * 100, 3),
            'support': None
        })
    
    # Создаем DataFrame
    df = pd.DataFrame(data)
    df.index = [''] * len(df)

    # Выводим таблицу
    print(df.to_string(index=False))

In [9]:
def save_contrel_date():
    pass

def add_token():
    pass


def use_CyclicLR():
    pass

In [10]:

lr_m = 1e-6
# lr = 1e-6 началось обучение, уменьшение TrainError
# добавить волидацию по шагам в течение эпохи обучения, а не каджые N епох
# Добавить сохранение ошибки в файлы для построения графиков
# проверить методы передачи сущьности (параетром, +тип, или в тексте выделяя тегами).
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", type=str, default="./ruRoberta-large/")
    #parser.add_argument("--model_name", type=str, default="sberbank-ai/ruRoberta-large", help="Имя модели")
    parser.add_argument("--max_seq_len", type=int, default=512) #128
    parser.add_argument("--batch_size", type=int, default=16)
    parser.add_argument("--epochs", type=int, default=15)
    parser.add_argument("--lr", type=float, default=lr_m)
    parser.add_argument("--init_checkpoint", type=str, default=None)
    parser.add_argument("--train_data", type=str, required=False, default="./data/train_data.csv")
    parser.add_argument("--validation_data", type=str, required=False, default="./data/validation.csv")
    parser.add_argument("--eval_data", type=str, required=False, default="./data/test.csv")
    parser.add_argument("--result", type=str, default=f"./result_lr_{lr_m}_ENS_Teg_21_multi_dropout_V2_T_NotToken/")
    # Игнорируем аргументы Jupyter
    args, unknown = parser.parse_known_args()
    os.makedirs(args.result, exist_ok=True)
    return args


if __name__ == "__main__":
    args = parse_args()
    print(args)  # Проверяем аргументы

    base_lr = args.lr
    min_lr = base_lr * (1/4)
    step_size_up = 400 #400

    best_avg_f1 = 0.0
    best_model_path = os.path.join(args.result, "best_model.pth")

    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    
    logging.basicConfig(level=logging.INFO)

    tokenizer = RobertaTokenizer.from_pretrained(args.model_name)
    #model = RobertaForSequenceClassification.from_pretrained(args.model_name, num_labels=3)
    model = RobertaWithMultiSampleDropout(model_name='./ruRoberta-large', num_labels=3, use_multi_sample_dropout=True )


    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    

    train_dataset = load_data(args.train_data, tokenizer, args.max_seq_len)
    validation_data = load_data(args.validation_data, tokenizer, args.max_seq_len)
    eval_dataset = load_data(args.eval_data, tokenizer, args.max_seq_len)
    # Извлекаем первые 10 примеров из датасета
    for i in range(2):
        input_ids, attention_mask, label = train_dataset[i]
        decoded_text = tokenizer.decode(input_ids, skip_special_tokens=False)
        print(f"Text: {decoded_text}")
        print(f"Label: {label.item()}")
        print("-" * 50)
    
    if train_dataset is None or eval_dataset is None or validation_data is None:
        sys.exit(f"Ошибка загрузки данных: убедитесь, что файлы {args.train_data}, {args.validation_data} и {args.eval_data} существуют и содержат нужные колонки.")

    # Получаем метки классов из тренировочного датасета
    train_labels = [label.item() for _, _, label in train_dataset]

    # Вычисляем веса классов
    class_weights = compute_ens_weights(train_labels, beta=0.999)

    # Вывод весов классов
    print(f"class_weights = {class_weights}")
    
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    validation_loader = DataLoader(validation_data, batch_size=args.batch_size)
    eval_loader = DataLoader(eval_dataset, batch_size=args.batch_size)
    
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    #scheduler = CyclicLR(optimizer, base_lr=min_lr, max_lr = base_lr, step_size_up = step_size_up, mode="triangular2",cycle_momentum=False )

    loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(device))
    
    def train():
        model.train()
        batches_per_validation = (len(train_loader) // 2)+2
        batches_per_test = (len(train_loader) // 5)+2
        
        for epoch in range(args.epochs):
            print(f"Epoch: {epoch}/{args.epochs}")
            total_loss = 0
            for i,batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1} - Batches")):
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                optimizer.zero_grad()
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = loss_fct(outputs.logits, labels)
                loss.backward()
                optimizer.step()
                #scheduler.step()
                
                total_loss += loss.item()
                if(i % batches_per_validation == 0 and i >50):
                    evaluate(epoch, backprop="Validation", vall_train = True)
                if(i % batches_per_test == 0 and i >50):
                    evaluate(epoch, backprop="Test", flag_print = True)
                
            logging.info(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Step: {len(train_loader)}")
            # Валидация с обратным распространением ошибки каждые 2 эпохи
            evaluate(epoch , backprop="Train", flag_print = True)
            evaluate(epoch , backprop="Test", flag_print = True)
            evaluate(epoch , backprop="Validation", flag_print = True, vall_train = True)
            
            
    
    def evaluate(epoch = None, backprop = "None", flag_print = False, vall_train = False):
        global best_avg_f1
        model.eval()
        all_preds, all_labels = [], []
        total_loss = 0
        print(f"evaluate, backprop: {backprop}")
        loader = validation_loader
        if backprop == "Validation":
            loader = validation_loader
        elif backprop == "Test":
            loader = eval_loader
        elif backprop == "Train":
            loader = train_loader
        
        else: loader = eval_loader
        with torch.no_grad() if not backprop == "Validation" else torch.enable_grad():  # Включаем градиенты для обучения на валидации
            for batch in loader:
            #for batch in loader:
                input_ids, attention_mask, labels = [x.to(device) for x in batch]
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = loss_fct(outputs.logits, labels)
                
                
                
                if backprop == "Validation" and vall_train:
                    #уменьшая тк обучающие данные удвоились
                    loss = loss
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    #scheduler.step()
                    loss = loss

                preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
                total_loss += loss.item()
                
        avg_loss = total_loss / len(loader)
        f1 = f1_score(all_labels, all_preds, average='macro')
        f1_pn = f1_score(all_labels, all_preds, labels=[1, 2], average='macro')
        avg_f1 = (f1 + f1_pn) / 2

        if(flag_print):
            print()
            print("--"*20)
            print(f"Result evaluate in {backprop}")
            #logging.info(f"{backprop} Loss: {avg_loss:.4f}")
            print(f"Loss: {avg_loss:.4f}")
            print(f"F1-macro: {f1:.4f}")
            print(f"F1-pn: {f1_pn:.4f}")
            #print(classification_report(all_labels, all_preds))
            castom_classification_report(all_labels, all_preds)

        # 💾 Сохраняем модель только при улучшении средней метрики
        if (avg_f1 > best_avg_f1) and backprop == "Test":
            best_avg_f1 = avg_f1
            torch.save(model.state_dict(), best_model_path)
            tokenizer.save_pretrained(os.path.join(args.result, "tokenizer"))
            
            print(f"[Checkpoint] 🎯 Новый лучший средний F1: {best_avg_f1:.4f}")
            print(f"[Checkpoint] 💾 Модель сохранена: {best_model_path}")
            # 🔸 Сохраняем значение лучшего F1 в файл
            best_score_path = os.path.join(args.result, "best_score.txt")
            save_metrics_best(epoch, all_labels, all_preds, args.result, backprop, avg_loss)
            
            save_confusion_matrix(epoch, all_labels, all_preds, args.result, backprop)
            with open(best_score_path, "w") as f:
                f.write(f"Epoch: {epoch}\n")
                f.write(f"Loss: {avg_loss:.4f}\n")
                f.write(f"F1-pn0: {f1:.4f}\n")
                f.write(f"F1-pn: {f1_pn:.4f}\n")
            print(f"Loss: {avg_loss:.4f}")
            print(f"F1-macro: {f1:.4f}")
            print(f"F1-pn: {f1_pn:.4f}")


        if epoch is not None:
            save_metrics(epoch, all_labels, all_preds, args.result, backprop, avg_loss)
            
        
    if args.init_checkpoint:
        model.load_state_dict(torch.load(args.init_checkpoint, map_location=device))

    train()

Namespace(model_name='./ruRoberta-large/', max_seq_len=512, batch_size=16, epochs=15, lr=1e-06, init_checkpoint=None, train_data='./data/train_data.csv', validation_data='./data/validation.csv', eval_data='./data/test.csv', result='./result_lr_1e-06_ENS_Teg_21_multi_dropout_V2_T_NotToken/')


Some weights of RobertaModel were not initialized from the model checkpoint at ./ruRoberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text: <s>Джеймс «Бадди» Макгирт (James (Buddy) McGirt, тренер Дадашева упрашивал дагестанского [ENTITY:PROFESSION] спортсмена [/ENTITY:PROFESSION] остановить бой, но тот хотел продолжать.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Label: 0
--------------------------------------------------
Text: <s>«За всю нашу долгую карьеру нам довелось играть во многих и

Epoch 1 - Batches:  20%|██        | 85/415 [00:50<03:15,  1.69it/s]

evaluate, backprop: Test

----------------------------------------
Result evaluate in Test
Loss: 0.7907
F1-macro: 0.2782
F1-pn: 0.0000
             label  precision  recall  f1-score  support
                 0     71.628 100.000    83.469    616.0
                 1      0.000   0.000     0.000    111.0
                 2      0.000   0.000     0.000    133.0
          accuracy     71.628     NaN       NaN      NaN
         macro avg     23.876  33.333    27.823    860.0
      weighted avg     51.306  71.628    59.787    860.0
avg f1 (class 1&2)        NaN     NaN     0.000      NaN


...


----------------------------------------
Result evaluate in Test
Loss: 0.6446
F1-macro: 0.7411
F1-pn: 0.6675
             label  precision  recall  f1-score  support
                 0     89.198  88.474    88.835    616.0
                 1     66.355  63.964    65.138    111.0
                 2     66.197  70.677    68.364    133.0
          accuracy     82.558     NaN       NaN      NaN
         macro avg     73.917  74.372    74.112    860.0
      weighted avg     82.693  82.558    82.610    860.0
avg f1 (class 1&2)        NaN     NaN    66.751      NaN


Epoch 10 - Batches: 100%|██████████| 415/415 [05:45<00:00,  1.20it/s]
INFO:root:Epoch 10, Loss: 0.025798580802539477, Step: 415


evaluate, backprop: Train

----------------------------------------
Result evaluate in Train
Loss: 0.0141
F1-macro: 0.9981
F1-pn: 0.9975
             label  precision  recall  f1-score  support
                 0     99.958  99.895    99.927   4774.0
                 1     99.650  99.766    99.708    856.0
                 2     99.703  99.901    99.802   1007.0
          accuracy     99.879     NaN       NaN      NaN
         macro avg     99.770  99.854    99.812   6637.0
      weighted avg     99.880  99.879    99.879   6637.0
avg f1 (class 1&2)        NaN     NaN    99.755      NaN
evaluate, backprop: Test

----------------------------------------
Result evaluate in Test
Loss: 0.6726
F1-macro: 0.7427
F1-pn: 0.6712
             label  precision  recall  f1-score  support
                 0     91.081  86.201    88.574    616.0
                 1     65.789  67.568    66.667    111.0
                 2     61.350  75.188    67.568    133.0
          accuracy     82.093     NaN       

Epoch 11 - Batches:  18%|█▊        | 76/415 [00:45<03:21,  1.68it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd

def load_and_display_metrics(metrics_file):
    # Загружаем CSV-файл
    df = pd.read_csv(metrics_file)
    df = df[df["backprop"] == "Test"]
    # Определяем количество эпох
    epochs = df["epoch"].unique()
    print(epochs)
    
    for epoch in epochs:
        # Фильтруем данные по эпохе
        epoch_df = df[df["epoch"] == epoch]
        
        for idx, row in epoch_df.iterrows():
            loss = row["loss"]
            backprop_value = row["backprop"]
            print(f"\nEpoch {epoch} (Backprop: {backprop_value}) (Loss: {loss})\n" + "-"*30)
            # Формируем таблицу в стиле classification_report
            table_data = {}
            class_labels = sorted(
                set(col.split("_")[1] for col in df.columns 
                    if col.startswith("class_") and "precision" in col)
            )
            
            # Фильтруем только числовые метки классов
            class_labels = [label for label in class_labels if label.isdigit()]
            
            for label in class_labels:
                table_data[int(label)] = {
                    "precision": row[f"class_{label}_precision"],
                    "recall": row[f"class_{label}_recall"],
                    "f1-score": row[f"class_{label}_f1"],
                }
            
            # Добавляем средние значения
            table_data["accuracy"] = {"precision": "", "recall": "", "f1-score": row["accuracy"] }
            table_data["macro avg"] = {
                "precision": row["macro_precision"],
                "recall": row["macro_recall"],
                "f1-score": row["macro_f1"],
            }
            table_data["weighted avg"] = {
                "precision": row["weighted_precision"],
                "recall": row["weighted_recall"],
                "f1-score": row["weighted_f1"],
            }
            
            # Выводим таблицу
            df_table = pd.DataFrame.from_dict(table_data, orient="index")
            print(df_table.to_string())

# Пример использования
metrics_file = "./result_lr_1e-06_test/metrics_class_weights.csv"
load_and_display_metrics(metrics_file)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузка CSV-файла
df = pd.read_csv("metrics_class_weights.csv")  # укажи путь к файлу, если он отличается

# Построение графика loss
plt.figure(figsize=(10, 6))

for stage in ['Train', 'Validation', 'Test']:
    stage_data = df[df['backprop'] == stage]
    plt.plot(stage_data['epoch'], stage_data['loss'], label=stage)

plt.title('Значение Loss по эпохам')
plt.xlabel('Эпоха')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
